In [19]:
import pandas as pd
from collections import Counter

In [20]:
df_sens = pd.read_csv("sensitive_topics_vs_source.csv")

In [21]:
Counter(df_sens['source'])

Counter({'toloka': 9291, 'trtest': 378, 'self': 1978, 'kw': 21656})

In [22]:
topics = ['offline_crime', 'online_crime', 'drugs', 'gambling', 'pornography',
       'prostitution', 'slavery', 'suicide', 'terrorism', 'weapons',
       'body_shaming', 'health_shaming', 'politics', 'racism', 'religion',
       'sexual_minorities', 'sexism', 'social_injustice']

In [23]:
df_sens.head()

,text,offline_crime,online_crime,drugs,gambling,pornography,prostitution,slavery,suicide,terrorism,weapons,body_shaming,health_shaming,politics,racism,religion,sexual_minorities,sexism,social_injustice,source
0,РРРРРРЯЯЯЯЯЯЯ РУССКИЕ ВАНЬКИ НИКОМУ НЕ НУЖНЫ!!...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,toloka
1,"По моему, быдло тот кто осуждает чужие предпоч...",0.0,0.0,0.0,0.0,1.0,0.80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,toloka
2,Бабы это вообще безхуйные сиськастые ссущие си...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,trtest
3,изнасиловалиа говорила что девственницаЯ так д...,0.0,0.0,0.0,0.0,1.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,trtest
4,"и что ? не сосут только уебищные тни, видосов ...",0.0,0.0,0.0,0.0,1.0,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,toloka


In [24]:
def get_self_check_df(target_df):
    collected_df_list = []
    for t in topics:
        df_c = target_df[target_df[t] > 0.8][:10]
        topics_curr = [t] * len(df_c)
        df_c['topic'] = topics_curr
        df_c = df_c[['text','topic']]
        collected_df_list.append(df_c)
    df_check = pd.concat(collected_df_list)      
    df_check.drop_duplicates(subset = ['text'], inplace = True)
    return df_check
    

In [25]:
df_sens_kw = df_sens[df_sens['source'] == 'kw']

In [26]:
df_sens_kw_for_check = get_self_check_df(df_sens_kw)

In [27]:
len(df_sens_kw_for_check), Counter(df_sens_kw_for_check['topic'])

(173,
 Counter({'offline_crime': 10,
          'online_crime': 10,
          'drugs': 10,
          'gambling': 10,
          'pornography': 10,
          'prostitution': 10,
          'slavery': 10,
          'suicide': 10,
          'terrorism': 10,
          'weapons': 10,
          'body_shaming': 10,
          'health_shaming': 10,
          'politics': 8,
          'racism': 8,
          'religion': 10,
          'sexual_minorities': 9,
          'sexism': 10,
          'social_injustice': 8}))

In [28]:
df_sens_kw_for_check.to_csv("check_keywords_lables.csv",index = None)

In [29]:
df_self = df_sens[df_sens['source'] == 'self']

In [30]:
df_self_for_check = get_self_check_df(df_self)

In [31]:
len(df_self_for_check), Counter(df_self_for_check['topic'])

(101,
 Counter({'offline_crime': 10,
          'online_crime': 10,
          'drugs': 10,
          'gambling': 1,
          'pornography': 10,
          'prostitution': 1,
          'slavery': 2,
          'suicide': 10,
          'terrorism': 2,
          'weapons': 4,
          'body_shaming': 1,
          'health_shaming': 10,
          'politics': 9,
          'sexual_minorities': 1,
          'sexism': 10,
          'social_injustice': 10}))

In [32]:
df_self_for_check.to_csv("check_self_lables.csv",index = None)

In [33]:
df_toloka_high_conf = pd.read_csv("TOLOKA_high_conf_for_verification.csv")

In [34]:
df_toloka_high_conf_for_check = get_self_check_df(df_toloka_high_conf)

In [35]:
len(df_toloka_high_conf_for_check), Counter(df_toloka_high_conf_for_check['topic'])

(158,
 Counter({'offline_crime': 10,
          'online_crime': 10,
          'drugs': 10,
          'gambling': 6,
          'pornography': 9,
          'prostitution': 8,
          'slavery': 9,
          'suicide': 10,
          'terrorism': 10,
          'weapons': 8,
          'body_shaming': 10,
          'health_shaming': 10,
          'politics': 8,
          'racism': 8,
          'religion': 7,
          'sexual_minorities': 8,
          'sexism': 8,
          'social_injustice': 9}))

In [36]:
df_toloka_high_conf_for_check.to_csv("check_toloka_high_conf_lables.csv", index = None)

# Восполнение полноты ключевых слов

In [37]:
import os
import pandas as pd

In [43]:
folder = './gr3_kw_self'
collected_df = []
for f in os.listdir(folder):
    if 'nokw' in f:
        topic = f.split("_")[:-1]
        pth = os.path.join(folder,f)
        df_c = pd.read_csv(pth)
        df_c = df_c.sample(150, random_state = 1)
        
        df_c['topic'] = ['_'.join(topic)] * len(df_c)
        collected_df.append(df_c)
        
collected_df = pd.concat(collected_df)


In [44]:
collected_df.head()

,processed,topic
6317,Удлинители бывают различных видов: металличес...,halth_shaming
27538,1 степень - способность к общению со снижение...,halth_shaming
99732,Просто получается что любви то у тебя и нет и...,halth_shaming
33892,Ну если ты за жрал гавно и кормил комаров то...,halth_shaming
96338,... или я уже ничего не понимаю.,halth_shaming


In [46]:
collected_df.to_csv("no_keyword_spec_forums_for_self_label.csv", index = None)